In [ ]:
import numpy as np
import pandas as pd
import sqlite3 as sql
from sklearn.preprocessing import MinMaxScaler
from ipywidgets import interact ## para análisis interactivo
from sklearn import neighbors ### basado en contenido un solo producto consumido
import joblib
#### conectar_base_de_Datos

conn=sql.connect('data\\db_books2')
cur=conn.cursor()

#### ver tablas disponibles en base de datos ###

cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
cur.fetchall()

In [ ]:
######################################################################
################## 1. sistemas basados en popularidad ###############
#####################################################################

In [ ]:
##### recomendaciones basado en popularidad ######
## peliculas vistas por más de 20 personas
pd.read_sql("""SELECT movie_title,
                avg(rating) as avg_rat,
                count(*) as movie_num
                FROM full_ratings
                WHERE rating <> 0
                GROUP BY movie_title
                HAVING count(*) >= 20
                ORDER BY avg_rat DESC, movie_num DESC
                LIMIT 10""", conn)

In [ ]:
#### los mejores calificados por año publicacion ###
pd.read_sql("""SELECT Año, movie_title, num_vistas
                FROM (
                    SELECT
                        strftime('%Y', fecha_nueva) AS Año,
                        movie_title,
                        ROW_NUMBER() OVER(PARTITION BY strftime('%Y', fecha_nueva) ORDER BY COUNT(*) DESC) AS row_num,
                        MAX(COUNT(*)) OVER(PARTITION BY strftime('%Y', fecha_nueva)) AS num_vistas
                    FROM full_ratings
                    GROUP BY strftime('%Y', fecha_nueva), movie_title) AS subquery
                WHERE row_num = 1
                ORDER BY Año;""", conn)